In [1]:
# Importing the relevant packages
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorboard.plugins.hparams import api as hp

C:\Users\muham\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Downloading and preprocessing the data

In [2]:
# Defining some constants/hyperparameters
BUFFER_SIZE = 70_000 # for reshuffling
BATCH_SIZE = 128
NUM_EPOCHS = 20

In [3]:
# Downloading the MNIST dataset
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

In [4]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

In [5]:
# Creating a function to scale our data
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.

    return image, label

In [6]:
# Scaling the data
train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

In [7]:
# Defining the size of the validation set
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

In [8]:
# Defining the size of the test set
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

In [9]:
# Reshuffling the dataset
train_and_validation_data = train_and_validation_data.shuffle(BUFFER_SIZE)

In [10]:
# Splitting the dataset into trainig + validation
train_data = train_and_validation_data.skip(num_validation_samples)
validation_data = train_and_validation_data.take(num_validation_samples)

In [11]:
# Batching the data
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = mnist_test.map(scale).batch(num_test_samples)

## Defining hyperparameters

In [12]:
# Defining the hypermatarest we would test and their range
HP_FILTER_SIZE = hp.HParam('filter_size', hp.Discrete([3,5,7]))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

METRIC_ACCURACY = 'accuracy'

# Logging setup info
with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
    hp.hparams_config(
        hparams=[HP_FILTER_SIZE, HP_OPTIMIZER],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
    )

## Cerating functions for training our model and for logging purposes

In [13]:
# Wrapping our model and training in a function
def train_test_model(hparams):
    
    # Outlining the model/architecture of our CNN
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(50, hparams[HP_FILTER_SIZE], activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(50, hparams[HP_FILTER_SIZE], activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)), 
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(10)
    ])
    
    # Defining the loss function
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

    # Compiling the model with parameter value for the optimizer
    model.compile(optimizer=hparams[HP_OPTIMIZER], loss=loss_fn, metrics=['accuracy'])
    
    # Defining early stopping to prevent overfitting
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor = 'val_loss',
        mode = 'auto',
        min_delta = 0,
        patience = 2,
        verbose = 0, 
        restore_best_weights = True
    )
    
    # Training the model
    model.fit(
        train_data, 
        epochs = NUM_EPOCHS,
        callbacks = [early_stopping],
        validation_data = validation_data,
        verbose = 2
    )
    
    _, accuracy = model.evaluate(test_data)
    
    return accuracy

In [14]:
# Creating a function to log the resuls
def run(log_dir, hparams):
    
    with tf.summary.create_file_writer(log_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

## Training the model with the different hyperparameters

In [15]:
# Performing a grid search on the hyperparameters we need to test
session_num = 0

for filter_size in HP_FILTER_SIZE.domain.values:
    for optimizer in HP_OPTIMIZER.domain.values:
    
        hparams = {
            HP_FILTER_SIZE: filter_size,
            HP_OPTIMIZER: optimizer
        }
        run_name = "run-%d" % session_num
        print('--- Starting trial: %s' % run_name)
        print({h.name: hparams[h] for h in hparams})
        run('logs/hparam_tuning/' + run_name, hparams)

        session_num += 1

--- Starting trial: run-0
{'filter_size': 3, 'optimizer': 'adam'}



Epoch 1/20



422/422 - 14s - loss: 0.2872 - accuracy: 0.9166 - val_loss: 0.0932 - val_accuracy: 0.9690 - 14s/epoch - 34ms/step
Epoch 2/20
422/422 - 12s - loss: 0.0768 - accuracy: 0.9771 - val_loss: 0.0589 - val_accuracy: 0.9813 - 12s/epoch - 28ms/step
Epoch 3/20
422/422 - 12s - loss: 0.0567 - accuracy: 0.9827 - val_loss: 0.0413 - val_accuracy: 0.9872 - 12s/epoch - 28ms/step
Epoch 4/20
422/422 - 12s - loss: 0.0456 - accuracy: 0.9859 - val_loss: 0.0279 - val_accuracy: 0.9923 - 12s/epoch - 28ms/step
Epoch 5/20
422/422 - 12s - loss: 0.0387 - accuracy: 0.9884 - val_loss: 0.0296 - val_accuracy: 0.9908 - 12s/epoch - 28ms/step
Epoch 6/20
422/422 - 12s - loss: 0.0333 - accuracy: 0.9898 - val_loss: 0.0283 - val_accuracy: 0.9917 - 12s/epoch - 28ms/step
1/1 [==============================] - 1s 1s/step - loss: 0.0361 - accuracy: 0.9876
--- Starting trial: run-1
{'filter_size': 3, 'optimizer': 'sgd'}
Epoch 1/20
422/422 - 13s - loss: 1.4416 - accuracy: 0.6242 - val_loss: 0.4938 - val_accuracy: 0.8648 - 13s/epoch

In [16]:
import os
os.environ['TENSORBOARD_BINARY'] = 'C:\\Users\\muham\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages'

In [17]:
alias tensorboard='python3 -m tensorboard.main'

## Visualizing the hyperparameter results with Tensorboard

In [18]:
# Loading the Tensorboard extension
%load_ext tensorboard


In [19]:
%reload_ext tensorboard

In [20]:
%tensorboard --logdir "logs/hparam_tuning"

ERROR: Failed to start 'C:\\Users\\muham\\AppData\\Local\\Packages\\Py
thonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-
packages\\Python311\\site-packages' (set by the `TENSORBOARD_BINARY`
environment variable): [WinError 5] Access is denied

In [21]:
%locate tensorboard

UsageError: Line magic function `%locate` not found.
